In [16]:
import sys
import numpy as np
import pandas as pd

### Extract Minimum Free Energy Path as a function of s(R) from the free-energy surface obtained from the metadynamics simulations that is a function of both s(R) and z(R)

Note that min S and max S are the s(R) values of the first frame and final frame included in the reference path for performing metadynamics simulations. minS_actual and maxS_actual are the minimum and maximum s(R) values observed in the metadynamics simulations.

In [17]:
minS=1.0
maxS=114.0
minS_actual=1.0
maxS_actual=112

Read the free energies obtained at the end of the simulations and filter out any s(R) values that are greater than maxS_actual

In [18]:
data=pd.read_csv("fes.dat",header=None,sep="\s+",skiprows=9,usecols=[i for i in range(3)])
data.columns = ["s(R)","z(R)", f"$\Delta$G"]
data=data[(data["s(R)"]>minS_actual) & (data["s(R)"]<maxS_actual) & (data["z(R)"]>0.0)]

Convert z(R) from nm$^2$ to Å$^2$ and free energies from kJ/mol to kcal/mol

In [19]:
data["z(R)"]=data["z(R)"]*100
data[f"$\Delta$G"]=data[f"$\Delta$G"]/4.2

Divide s(R) data into 25 equally sized bins and within the each bin extract the minimum free energy (G)

In [20]:
bins=pd.cut(data["s(R)"],bins=25)
g=data["$\Delta$G"].groupby(bins).min()

For minimum G in each bin, extract the associated s(R) and z(R) values

In [21]:
z=np.array([data[data["$\Delta$G"]==i]['z(R)'] for i in g])
s=np.array([data[data["$\Delta$G"]==i]['s(R)'] for i in g])

 Store the extracted minimum G values and associated s(R) and z(R) values in a file named mfep-profile-np.dat

In [22]:
gprofile=pd.DataFrame(columns=["s(R)","z(R)","G"],index=[i for i in range(1,26)])
gprofile["G"]=g.unique().round(1)
gprofile["s(R)"]=s.round(1)
gprofile["z(R)"]=z.round(1)
gprofile.to_csv("mfep-profile-np.dat",sep="\t",index=None,float_format="%5.1f")